In [1]:
import os
from crate import client
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from tensorflow import keras
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
import plotly.graph_objects as go
from tensorflow.keras.models import Sequential, Model
from pickle import load

In [2]:
connection = client.connect("http://localhost:4200/", username="crate")
cursor= connection.cursor()

In [13]:
cursor.execute("SELECT * FROM time ORDER BY timestamps DESC ")
query=cursor.fetchmany(2922)
df=pd.DataFrame(query, columns=['timestamps', 'value'])
df['dates']=pd.to_datetime(df['timestamps'], unit='ms')
df.index=df['dates']

In [14]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=df['dates'], y=df['value'], name='values'))
fig.update_layout(showlegend=True, title='data')
fig.show()

In [15]:
scaler=load(open('scaler.pkl', 'rb'))
TIME_STEPS=10
def create_sequences(X, y, time_steps=TIME_STEPS):
    Xs, ys = [], []
    for i in range(len(X)-time_steps):
        Xs.append(X.iloc[i:(i+time_steps)].values)
        ys.append(y.iloc[i+time_steps])
    
    return np.array(Xs), np.array(ys)
df['value']=scaler.transform(df[['value']])
test_x,test_y=create_sequences(df[['value']],df['value'])

In [16]:
model=keras.models.load_model('model')

In [19]:
threshold=0.18149378544428071
pred=model.predict(test_x)
loss=np.mean(np.abs(pred - test_x), axis=1)
df_=df[TIME_STEPS-1:-1]
df_['loss']=loss
df_['threshold']=2*threshold
df_['anomaly_detected']=df_['loss']>df_['threshold']

anomalies = df_.loc[df_['anomaly_detected'] == True]

fig = go.Figure()
fig.add_trace(go.Scatter(x=df_['dates'], y=df_['loss'], name='Test loss'))
fig.add_trace(go.Scatter(x=df_['dates'], y=df_['threshold'], name='Threshold'))
fig.update_layout(showlegend=True, title='Test loss vs. Threshold')
fig.show()


<ipython-input-19-d39e275162e0>:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-19-d39e275162e0>:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-19-d39e275162e0>:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [20]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=df_['dates'], y=scaler.inverse_transform(df_['value']), name='Close price'))
fig.add_trace(go.Scatter(x=anomalies['dates'], y=scaler.inverse_transform(anomalies['value']), mode='markers', name='Anomaly'))
fig.update_layout(showlegend=True, title='Detected anomalies')
fig.show()

In [13]:
from sqlalchemy import create_engine

# create sqlalchemy engine
engine = create_engine('crate://localhost:4200')

In [37]:
df_=df_[['value', 'loss', 'threshold', 'anomaly_detected']]

In [ ]:
df_.to_sql('predicted', con = engine, if_exists = 'append', chunksize = 10000)